In [ ]:
import marimo as mo

## Creating tools

In [ ]:
from langchain.tools import tool

### Web search

In [ ]:
from ddgs import DDGS

In [ ]:
DDGSResult = list[dict[str, str]]

@tool
def search(
    query: str,
    max_results: int = 10,
    region: str = "us-en",
    safesearch: str = "moderate",
    timelimit: str | None = None,
    page: int = 1,
    backend: str = "auto",
) -> DDGSResult | str:
    """
    DuckDuckGo text search for web pages, articles, and information.

    Args:
        query: text search query.
        region: us-en, uk-en, ru-ru, etc. Defaults to us-en.
        safesearch: on, moderate, off. Defaults to "moderate".
        timelimit: d, w, m, y. Defaults to None.
        max_results: maximum number of results. Defaults to 10.
        page: page of results. Defaults to 1.
        backend: A single or comma-delimited backends. Defaults to "auto".
    Returns:
        List of dictionaries with search results.
    """
    results = DDGS().text(
        query=query,
        region=region,
        safesearch=safesearch,
        timelimit=timelimit,
        max_results=max_results,
        page=page,
        backend=backend,
    )
    return results

### Arxiv search

In [ ]:
import arxiv

In [ ]:
@tool
def arxiv_search(query: str, max_results: int = 3) -> list[dict]:
    """Search for relevant papers using the arXiv tool"""
    search = arxiv.Search(
        query=query, max_results=max_results, sort_by=arxiv.SortCriterion.Relevance
    )

    papers = []
    for result in search.results():
        papers.append(
            {
                "title": result.title,
                "authors": [author.name for author in result.authors],
                "summary": result.summary,
                "published": result.published.isoformat(),
                "url": result.entry_id,
                "arxiv_id": result.entry_id.split("/")[-1],
            }
        )

    return papers

## Pydantic models and State

In [ ]:
import operator
from textwrap import dedent
from typing import Annotated

from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field

In [ ]:
class ResearchQuery(BaseModel):
    topic: str = Field(..., description="Research topic to investigate")
    max_papers: int = Field(
        default=5, description="Maximum number of arXiv papers to retrieve"
    )
    max_web_results: int = Field(
        default=5, description="Maximum number of web results to retrieve"
    )
    year_filter: str | None = Field(
        default=None, description="Optional year filter for papers"
    )

class ResearchPlan(BaseModel):
    arxiv_query: str = Field(..., description="Formatted query for arXiv search")
    web_queries: list[str] = Field(..., description="List of web search queries")
    focus_areas: list[str] = Field(
        ..., description="Key focus areas for the research"
    )

class ArxivFinding(BaseModel):
    title: str = Field(..., description="Title of the paper")
    authors: list[str] = Field(..., description="List of author names")
    summary: str = Field(..., description="Summary of the paper")
    url: str = Field(..., description="ArXiv URL")
    published: str = Field(..., description="Publication date")

class ArxivFindings(BaseModel):
    papers: list[ArxivFinding] = Field(
        default_factory=list, description="List of found arXiv papers"
    )

class WebFinding(BaseModel):
    title: str = Field(..., description="Title of the web source")
    url: str = Field(..., description="URL of the web source")
    content_summary: str = Field(..., description="Summary of the content")

class WebFindings(BaseModel):
    sources: list[WebFinding] = Field(
        default_factory=list, description="List of found web sources"
    )

class ResearchReport(BaseModel):
    topic: str = Field(..., description="Research topic")
    key_findings: list[str] = Field(..., description="List of key findings")
    summary: str = Field(..., description="Overall summary of the research")
    gaps_identified: list[str] = Field(
        default_factory=list, description="Identified research gaps"
    )

class ReviewFeedback(BaseModel):
    approved: bool = Field(..., description="Whether the report is approved")
    missing_aspects: list[str] = Field(
        default_factory=list, description="Missing aspects in the report"
    )
    quality_score: float = Field(..., description="Quality score (0.0-1.0)")
    suggestions: list[str] = Field(
        default_factory=list, description="Suggestions for improvement"
    )

class ResearchState(BaseModel):
    query: ResearchQuery
    plan: ResearchPlan | None = None
    arxiv_findings: Annotated[list[ArxivFinding], operator.add] = Field(
        default_factory=list
    )
    web_findings: Annotated[list[WebFinding], operator.add] = Field(
        default_factory=list
    )
    report: ResearchReport | None = None
    review: ReviewFeedback | None = None
    errors: Annotated[list[str], operator.add] = Field(default_factory=list)
    iteration: int = 0

## Graph

In [ ]:
from langchain.agents import create_agent
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI
from langgraph.graph import END, StateGraph
from langgraph.types import RetryPolicy

In [ ]:
import os

from dotenv import find_dotenv, load_dotenv

load_dotenv(find_dotenv(usecwd=True))

BASE_URL = os.getenv("OPENAI_BASE_URL", "http://a6k2.dgx:34000/v1")
API_KEY = os.getenv("OPENAI_API_KEY", "")
MODEL_NAME = os.getenv("MODEL_NAME", "qwen3-32b")

In [ ]:
llm = ChatOpenAI(
    base_url=BASE_URL,
    api_key=API_KEY,
    model=MODEL_NAME,
)

### Prompts

In [ ]:
# Create parsers
planner_parser = PydanticOutputParser(pydantic_object=ResearchPlan)
arxiv_parser = PydanticOutputParser(pydantic_object=ArxivFindings)
web_parser = PydanticOutputParser(pydantic_object=WebFindings)
synthesizer_parser = PydanticOutputParser(pydantic_object=ResearchReport)
reviewer_parser = PydanticOutputParser(pydantic_object=ReviewFeedback)

# Planner prompt
PLANNER_PROMPT = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            dedent(
                """
        You are a research coordinator. Your task is to analyze the research topic and create a structured research plan.

        Based on the user's topic, generate:
        1. A well-formulated arXiv query
        2. Multiple web search queries to cover different aspects
        3. Key focus areas for the research
        4. Expected number of sources

        Be specific and thorough in your planning.

        {format_instructions}
        /no_think
    """
            ).strip(),
        ),
        (
            "human",
            dedent(
                """
        Topic: {topic}
        Max papers: {max_papers}
        Max web results: {max_web_results}
    """
            ).strip(),
        ),
    ]
)

# ArXiv researcher prompt (for agent)
ARXIV_SEARCH_PROMPT = dedent(
    """
    You are an academic researcher specializing in analyzing arXiv papers.

    Your task is to:
    1. Search for relevant papers using the arXiv tool
    2. Extract key information from each paper
    3. Summarize the main contributions
    4. Identify the most important papers for the research topic

    Focus on recent, high-quality publications. Use the available tools to search arXiv.
"""
).strip()

# ArXiv parser prompt
ARXIV_PARSER_PROMPT = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            dedent(
                """
        You are a helpful assistant that extracts paper information from agent search results.
        {format_instructions}
        /no_think
    """
            ).strip(),
        ),
        (
            "human",
            dedent(
                """
        Extract papers from these agent results:

        {agent_output}
    """
            ).strip(),
        ),
    ]
)

# Web researcher prompt (for agent)
WEB_SEARCH_PROMPT = dedent(
    """
    You are a web researcher specializing in finding and analyzing online sources.

    Your task is to:
    1. Search the web using the DuckDuckGo tool
    2. Extract and summarize relevant content
    3. Assess the relevance and quality of each source
    4. Provide concise summaries of key findings

    Focus on authoritative and up-to-date sources. Use the available tools to search the web.
"""
).strip()

# Web parser prompt
WEB_PARSER_PROMPT = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            dedent(
                """
        You are a helpful assistant that extracts web source information from agent search results.
        {format_instructions}
        /no_think
    """
            ).strip(),
        ),
        (
            "human",
            dedent(
                """
        Extract web sources from these agent results:

        {agent_output}
    """
            ).strip(),
        ),
    ]
)

# Synthesizer prompt
SYNTHESIZER_PROMPT = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            dedent(
                """
        You are a research analyst and technical writer.

        Your task is to synthesize all collected research into a comprehensive report.

        You should:
        1. Identify and summarize key findings across all sources
        2. Write a coherent summary of the research landscape
        3. Highlight important papers and sources
        4. Identify gaps in current research or understanding

        Create a well-structured, informative research report.

        {format_instructions}
        /no_think
    """
            ).strip(),
        ),
        (
            "human",
            dedent(
                """
        {context}
    """
            ).strip(),
        ),
    ]
)

# Reviewer prompt
REVIEWER_PROMPT = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            dedent(
                """
        You are a critical reviewer and research quality assessor.

        Your task is to evaluate the research report for:
        1. Completeness - are all important aspects covered?
        2. Quality - is the analysis thorough and well-reasoned?
        3. Clarity - is the report well-written and understandable?
        4. Missing aspects - what could be improved or added?

        Provide a quality score (0.0-1.0) and decide if the report is approved or needs revision.
        Be strict but fair in your assessment.

        {format_instructions}
        /no_think
    """
            ).strip(),
        ),
        (
            "human",
            dedent(
                """
        {report_text}
    """
            ).strip(),
        ),
    ]
)

### Structure

In [ ]:
arxiv_agent = create_agent(llm, [arxiv_search], system_prompt=ARXIV_SEARCH_PROMPT)
web_agent = create_agent(llm, [search], system_prompt=WEB_SEARCH_PROMPT)

In [ ]:
async def planner_node(state):
    messages = PLANNER_PROMPT.format_messages(
        format_instructions=planner_parser.get_format_instructions(),
        topic=state.query.topic,
        max_papers=state.query.max_papers,
        max_web_results=state.query.max_web_results,
    )
    response = await llm.ainvoke(messages)
    plan = planner_parser.parse(response.content)
    return {"plan": plan}

In [ ]:
async def arxiv_researcher_node(state):
    if not state.plan:
        raise ValueError("No research plan available")

    query = state.plan.arxiv_query
    max_papers = state.query.max_papers

    message = HumanMessage(
        content=f"Search arXiv for: '{query}'. Find up to {max_papers} relevant papers."
    )
    agent_response = await arxiv_agent.ainvoke({"messages": [message]})
    agent_output = agent_response["messages"][-1].content

    chain = ARXIV_PARSER_PROMPT | llm | arxiv_parser
    parsed_result = await chain.ainvoke(
        {
            "format_instructions": arxiv_parser.get_format_instructions(),
            "agent_output": agent_output,
        }
    )

    return {"arxiv_findings": parsed_result.papers}

In [ ]:
async def web_researcher_node(state):
    if not state.plan:
        raise ValueError("No research plan available")

    queries = " AND ".join(state.plan.web_queries)
    max_results = state.query.max_web_results

    message = HumanMessage(
        content=f"Search web for: {queries}. Find up to {max_results} relevant sources."
    )
    agent_response = await web_agent.ainvoke({"messages": [message]})
    agent_output = agent_response["messages"][-1].content

    chain = WEB_PARSER_PROMPT | llm | web_parser
    parsed_result = await chain.ainvoke(
        {
            "format_instructions": web_parser.get_format_instructions(),
            "agent_output": agent_output,
        }
    )

    return {"web_findings": parsed_result.sources}

In [ ]:
async def synthesizer_node(state):
    arxiv_summary = "\n".join(
        [
            f"- {p.title} by {', '.join(p.authors)}: {p.summary}"
            for p in state.arxiv_findings
        ]
    )
    web_summary = "\n".join(
        [f"- {s.title} ({s.url}): {s.content_summary}" for s in state.web_findings]
    )

    feedback = ""
    if state.review and state.review.suggestions:
        feedback = "Previous review feedback: " + ", ".join(
            state.review.suggestions
        )

    context = f"""Topic: {state.query.topic}

ArXiv Papers:
{arxiv_summary if arxiv_summary else "No papers found"}

Web Sources:
{web_summary if web_summary else "No sources found"}

{feedback}"""

    messages = SYNTHESIZER_PROMPT.format_messages(
        format_instructions=synthesizer_parser.get_format_instructions(),
        context=context,
    )
    response = await llm.ainvoke(messages)
    report = synthesizer_parser.parse(response.content)
    return {"report": report, "iteration": state.iteration + 1}

In [ ]:
async def reviewer_node(state):
    if not state.report:
        raise ValueError("No report available to review")

    report_text = f"""Report Summary: {state.report.summary}
Key Findings: {", ".join(state.report.key_findings)}
Number of ArXiv papers: {len(state.arxiv_findings)}
Number of Web sources: {len(state.web_findings)}
Gaps identified: {", ".join(state.report.gaps_identified)}"""

    messages = REVIEWER_PROMPT.format_messages(
        format_instructions=reviewer_parser.get_format_instructions(),
        report_text=report_text,
    )
    response = await llm.ainvoke(messages)
    review = reviewer_parser.parse(response.content)
    return {"review": review}

In [ ]:
def should_revise(state) -> str:
    if state.review and not state.review.approved and state.iteration < 2:
        return "synthesizer"
    return "end"

In [ ]:
g = StateGraph(ResearchState)

In [ ]:
retry_policy = RetryPolicy(
    max_attempts=3,
    initial_interval=0.5,
    backoff_factor=2.0,
    jitter=True,
)

g.add_node("planner", planner_node)
g.add_node("arxiv_researcher", arxiv_researcher_node, retry=retry_policy)
g.add_node("web_researcher", web_researcher_node, retry=retry_policy)
g.add_node("synthesizer", synthesizer_node)
g.add_node("reviewer", reviewer_node)

g.set_entry_point("planner")

g.add_edge("planner", "arxiv_researcher")
g.add_edge("planner", "web_researcher")

g.add_edge("arxiv_researcher", "synthesizer")
g.add_edge("web_researcher", "synthesizer")

g.add_edge("synthesizer", "reviewer")

g.add_conditional_edges(
    "reviewer", should_revise, {"synthesizer": "synthesizer", "end": END}
)

app = g.compile()

## Graph Visualization

In [ ]:
app.get_graph().draw_ascii()

b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01h\x00\x00\x02+\x08\x02\x00\x00\x00M\xa3\x94\xdd\x00\x00\x10\x00IDATx\x9c\xec\xdd\x07|\x13e\x1f\x07\xf0\xe7\x92tO\xcahiKYe\xef\xbdd\x83,Y2e\x892DE\xd9(\x82\xbeL\x91\x8d\x88,\x11\x15eJ\x11\x19"S@6\x82\x05Z@(-\xbb\xac\xee\xdd&w\xef?\xb9\x12B\x9b+M\xe9\xb8\xf4~_\xf9\xc4\xcb\xcd4\xb9\xfb\xe5\x19\x97;\x8d \x08\x0c\x00\xc0\x12\x1a\x06\x00`!\x04\x07\x00X\x0c\xc1\x01\x00\x16Cp\x00\x80\xc5\x10\x1c\x00`1\x04\x07\x00X\x0c\xc1a\xdd\xae_\x88\r\xb9\x18\x1f\xfb4--U\xe0y33\xa8m8]Z\xc6\x1ewN\xc5\xa9TL\xa7\xcd8^\xad\xe6t\xba\x17Fr\x1c\'\xe8\x1f\x99\xc0\xbf83\xc7T\x1c\xc7\x9b\x8c\xa4ur\x8c\xf1|\xc6u\xaaT\\\x86\x91j\rgc\xc7\xb9\x14Q\xfbUv\xaa\xde\xc4\x9d\x81\x15\xe2p\x1e\x875:\xfd\xc7\xd3\xab\xe7\xe2\x12cu4lc\xc3\xd4\xb6\x9c\xc6F\xcd\t\\\xe69U\x1a\xc6k3\x8d\xe5\x04\xfd\xf1\xac\xcb4\x9a\xd6\x91!8T\xb4\x83p\x86\xe0\xc84\xb3\xea\x85\x91\xf4\x94\x98\x99M\xcd\x04]\x86\x05\x05\x8a\'m\x9a\xa0M\xe5u:\xe6\xe0\xac*_\xd3\xb9e\xaf\x12\x0c\xac\x07\x82\xc3\xca\x1c\xff\xfdq\xd0\xc9X\xfa\xe0\xbc\xfc\xec\x1at\xf0\xf0\xf1wd\xd6\xec\xc9\xfd\xa4S{"\x1f\x86%\xf3:\xc1\xbf\x8es\xbb\xb7\xbc\x18X\x03\x04\x875Y\xf7Ehj\n_\xb3\xb9k\xd37\n\xdb\xf7\xf3\xf9\xc3\x11\x17\x0eES9h\xd8\xacr\x0cd\x0f\xc1a\x1db\x9e&\xfd<\xf7~)\x7f\xbb\xee\xef\x97b\x85\xd7\xbe\x9f\xc2C.%t\x18\xe2\xe9_\xcb\x85\x81\x8c!8\xac@j\xaan\xcd\'ao\x8c\xf4*S\xc5\x99\x15vq1)?M\xbf;lVY\x07g5\x03\xb9Bp\xc8\xdd\xe3\xbbI\xbf~}\xff\xc3\x85\xfeLIVL\x0ci\xd3\xafx\xe5\x06n\x0cdI\xc5@\xde~]z\xff\xcd\x0f\x15\xd7d\xf8\xc1B\xffC\x9b\x9e\xa4&\xa52\x90%\x04\x87\xac\xad\x9d\x16\xeaW\xd9\xa1d\xd9\xc2_C\xc9\xaczS\xd7\x1ff\xdce K\x08\x0e\xf9\xfa\xeb\xd7G:\x9e\xef:\xc2\x87)R\xcb\xde%\xd46\xdc\xae5\xf7\x19\xc8\x0f\x82C\xbe\xae\x9e\x8dk\xd0\xc1\x83)X\xbb\x81\xc5\xee\\Kb ?\x08\x0e\x99:\xb2\xed\x11=\xd6m\xa9\xe8\xe0(S\xd9\xd5\xc6\x9e\xdb\xf5\x1d\n\x1d\xb2\x83\xe0\x90\xa9\x1b\x17\xe3}+80\xc5\xabX\xdf\xf9\xe1\xadd\x062\x83\xe0\x90\xa9\x94x\xa1\xdd[\x9e,\x7f\xb5o\xdf\xfe\xfe}\x8b\xbf\xdeo\xde\xbc\xf9\xc6\x1bo\xb0\xbc\xd1\xeaM\xcf\xd4$!\xf2\t\xb2C^\x10\x1crt\xf6\xc0S\xb5\rst\xc9\xd7\xdf.\x87\x87\x87GEE1\xcb]\xb9r\x85\xe5%\xaa\xad\\8\x10\xcd@N\x10\x1cr\xf4\xe0z\xb2\xbdC^\x9d7)\x08\xc2\xc6\x8d\x1b\x07\x0c\x18\xd0\xacY\xb3A\x83\x06-_\xbe\\\xa7\xd3\xfd\xf3\xcf?]\xbbv\xa5\xa9\xdd\xbbw\x9f0a\x023\x94#\xe6\xcd\x9b\xd7\xbbw\xef\xa6M\x9b\xd2l\xdb\xb6m\x13\x17\x0f\t\t\xa9_\xbf\xfe\xf1\xe3\xc7;v\xec\xf8\xd6[o\xadZ\xb5j\xc6\x8c\x19\x0f\x1f>\xa4\x91\x1b6l`y\xc0\xd1Y\xfd\xf4\x01N\xe8\x90\x17\\\x8fC\x8e\xe2c\xd3\xec\x9d9\x9676o\xde\xbcn\xdd\xba\xb1c\xc7Rp\x1c9r\xe4\xdbo\xbfurrz\xe7\x9dw\x96.]J#\x7f\xff\xfdw\x1f\x1f}\x07\xf0\xa2E\x8b\x1e<x0u\xeaT\x8e\xe3n\xdd\xbaE!R\xb2dIZ\xc4\xc6\xc6\x86\xa6\xae]\xbbv\xf0\xe0\xc1\xb5k\xd7\xaeV\xadZjj\xea\xfe\xfd\xfbw\xef\xde\xcd\xf2\x86\xb3\xbb:\xfaI\x1a\x039Ap\xc8\x11\xaf\xe5\xec\xf3\xec\x97\x1a\x17.\\\xa8Z\xb5\xaa\xd8*\xd1\xb3g\xcf\x06\r\x1a$&&f\x9em\xee\xdc\xb9\t\t\t\xde\xde\xde4L\xa5\x89\x9d;w\x9e<y\x92\x82C\x7fq\x0e\xc6\x1a7n<p\xe0@\x96/\xec\x1dmt\xa9Z\x06r\x82\xe0\x90#A\x7fY\x9c\xbc*q\xd4\xaaU\xeb\x9bo\xbe\x999sf\x9d:uZ\xb4h\xe1\xeb\xebkv6\xaa\xd1P\xd9\xe4\xc4\x89\x13\xb7o\xdf\x16\xc7\x88%\x11Q\x95*UX~\xa1\xa4\xe2\x19~Q%/\x08\x0e9Ri\x84\xb44\x1d\xcb\x1b\xd4\xbaAu\x93\xa3G\x8fR\xdb\x84F\xa3\xa1\x9e\x94\x8f?\xfe\xb8x\xf1\xe2\xa6\xf3\xf0<?f\xcc\x18\xaa\x83\x8c\x1e=\x9a\x8a\x1b...\xc3\x86\r3\x9d\xc1\xce\xce\x8e\xe5\x97\xc4x\xadJ\x8d\xe0\x90\x17\x04\x87\x1c98\xaac\xa3\xf2\xaaV\xafR\xa9z\x1a\x84\x86\x86\x9e={v\xcd\x9a5\xf1\xf1\xf1K\x96,1\x9d\xe7\xda\xb5k\xc1\xc1\xc1+V\xach\xd8\xb0\xa18&..\xaeD\x89\x82\xb9zPB\x8c6\xef\x9a\x8a!g\xd0\xab"G%\xcb\xda\xa7&\xe5\xd5w,\xb5bR\x8f\t\r\x94+W\xae\x7f\xff\xfe\xd43\xf2\xdf\x7f\xffe\x98\':Z\xdf\xfdiL\x8aP\x03V@\x12b\xb5\xee^\xb6\x0c\xe4\x04\xc1!G\xcd\xba\x15\xd7\xa6\xe6Up\xfc\xf9\xe7\x9f\x93&M:v\xecXLL\x0c\xf5\xaa\x1e>|\x98Z=h|\x992e\xe8\xf1\xc0\x81\x03AAA\x94)T\x8b\xf9\xf9\xe7\x9fccc\xa9Ke\xc1\x82\x05\xd4\x1a\x1a\x1e\x1env\x85~~~O\x9f>\xa5\x0e\x1ackH\xeeJKa\xb5\x9a\xbb2\x90\x13\x04\x87L\xd9\xd8q\xfb~y\xc0\xf2\xc0\xb4i\xd3(\x17\xc6\x8f\x1f

## Demo Execution

In [ ]:
QUERY = "Multi-agent systems with LLMs"

In [ ]:
demo_query = ResearchQuery(
    topic=QUERY,
    max_papers=3,
    max_web_results=3,
)

In [ ]:
state = ResearchState(query=demo_query)
result = await app.ainvoke(state)

/tmp/marimo_80011/__marimo__cell_BYtC_.py:9: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  for result in search.results():


In [ ]:
result